Notebook to download and clean data (from snowflake) that needs to be shipped to Experian on a monthly basis to get it appended

In [1]:
import pandas as pd

import snowflake.connector

import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt 

from pydataset import data

from datetime import datetime, timedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [2]:
import config as cfg

In [3]:
snowflake_user = cfg.snowflake_access['snowflake_user']
snowflake_pass = cfg.snowflake_access['snowflake_pass']
snowflake_acct = cfg.snowflake_access['snowflake_acct']
snowflake_wh = 'BIRDFACTS_PROD_WAREHOUSE'
snowflake_db = 'BIRDFACTSDEV'
snowflake_schema = 'BIRDFACTS_ANALYTICS'

con = snowflake.connector.connect(
    user = snowflake_user,
    password = snowflake_pass,
    account = snowflake_acct,
    warehouse = snowflake_wh,
    database = snowflake_db,
    schema = snowflake_schema
)

cs = con.cursor()

### All customers broken down by product, timeline and New/Existing status

In [66]:
dashers_query = """
select * from
(
select 
case 
when happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
taxonomy_style,
is_new_customer,
sum(gross_sales_usd) as gross_sales,
count(distinct customer_id) as num_customers
from
fact_sales
where 
taxonomy_category = 'Shoes'
and
((happened_at_local_date between '2020-01-01' and '2020-05-16')
or (happened_at_local_date between '2019-09-18' and '2019-10-07')
or (happened_at_local_date between '2019-06-14' and '2019-07-03'))
and sales_channel = 'eCommerce'
and profit_center_country = 'United States'
group by 1,2,3
order by 1,2,3
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [67]:
dashers = pd.DataFrame(cs.execute(dashers_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [68]:
dashers['GROSS_SALES'] = pd.to_numeric(dashers['GROSS_SALES'])
dashers['NUM_CUSTOMERS'] = pd.to_numeric(dashers['NUM_CUSTOMERS'])

In [72]:
dashers['PRODUCT'] = np.where((dashers['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers['TIMELINE'] == 'Apr28-May16-2020')&(dashers['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers['TIMELINE'] == 'Apr28-May16-2020')&(dashers['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [80]:
dashers = dashers[~dashers['PRODUCT'].isin(['Rest'])]

In [103]:
dashers_new_pivot = dashers[dashers['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER'], 
                                 values = ['GROSS_SALES', 'NUM_CUSTOMERS'],
                                 aggfunc = 'sum')

In [104]:
dashers_old_pivot = dashers[dashers['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER'], 
                                 values = ['GROSS_SALES', 'NUM_CUSTOMERS'],
                                 aggfunc = 'sum')

In [110]:
dashers_pivot = pd.merge(dashers_new_pivot, dashers_old_pivot, how = 'left', on = ['TIMELINE', 'PRODUCT'])
dashers_pivot

GROSS_SALES NUM_CUSTOMERS  GROSS_SALES  \
IS_NEW_CUSTOMER                    True          True         False   
TIMELINE         PRODUCT                                              
Apr28-May16-2020 Dasher       1325000.00         10018   3941460.00   
                 Non-Dasher   3454587.21         36052   3856147.68   
Jan1-Apr27-2020  All         23078746.74        235253  22321304.07   
Jun14-Jul03-2019 Breezer       504640.00          4873   1201750.00   
Sep18-Oct07-2019 Mizzle       1606675.00         12452   2449925.00   

                            NUM_CUSTOMERS  
IS_NEW_CUSTOMER                     False  
TIMELINE         PRODUCT                   
Apr28-May16-2020 Dasher             25802  
                 Non-Dasher         35913  
Jan1-Apr27-2020  All               196601  
Jun14-Jul03-2019 Breezer            10079  
Sep18-Oct07-2019 Mizzle             17537

In [111]:
dashers_pivot.to_clipboard()

### All customers broken down by product, timeline and New/Existing status + MOSAIC

In [133]:
dashers_mosaic_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
a.is_new_customer,
left(c.mosaichousehold, 1) as mosaic_group,
count(distinct a.customer_id) as num_customers
from
fact_sales as a
left join
dim_customer as b
on a.customer_id = b.id
left join
experian_data as c
on b.email = c.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [134]:
dashers_mosaic = pd.DataFrame(cs.execute(dashers_mosaic_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [135]:
dashers_mosaic['NUM_CUSTOMERS'] = pd.to_numeric(dashers_mosaic['NUM_CUSTOMERS'])

In [136]:
dashers_mosaic['PRODUCT'] = np.where((dashers_mosaic['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_mosaic['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_mosaic['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_mosaic['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_mosaic['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_mosaic['TIMELINE'] == 'Apr28-May16-2020')&(dashers_mosaic['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_mosaic['TIMELINE'] == 'Apr28-May16-2020')&(dashers_mosaic['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [137]:
dashers_mosaic = dashers_mosaic[~dashers_mosaic['PRODUCT'].isin(['Rest'])]

In [138]:
mosaic_dictionary = pd.read_excel("experian_dictionary.xlsx", sheet_name = 'mosaichh_higher')

In [139]:
dashers_mosaic = pd.merge(dashers_mosaic, mosaic_dictionary, how = 'left', on = 'MOSAIC_GROUP')

In [140]:
dashers_mosaic_small = dashers_mosaic[dashers_mosaic['MOSAIC_GROUP'].isin(['A', 'C', 'O','G', 'B'])]

In [152]:
dashers_mosaic_new_pivot = dashers_mosaic_small[dashers_mosaic_small['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','MOSAIC'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [154]:
dashers_mosaic_old_pivot = dashers_mosaic_small[dashers_mosaic_small['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','MOSAIC'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [159]:
dashers_mosaic_pivot = pd.merge(dashers_mosaic_new_pivot, dashers_mosaic_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_mosaic_pivot
dashers_mosaic_pivot.to_clipboard()

IS_NEW_CUSTOMER                               True                        \
MOSAIC                      Booming with Confidence Flourishing Families   
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher                        1385                  763   
                 Non-Dasher                    4928                 2924   
Jan1-Apr27-2020  All                          31219                19912   
Jun14-Jul03-2019 Breezer                        633                  454   
Sep18-Oct07-2019 Mizzle                        1628                  970   

IS_NEW_CUSTOMER                                                                \
MOSAIC                      Power Elite Singles and Starters Young City Solos   
TIMELINE         PRODUCT                                                        
Apr28-May16-2020 Dasher            3367                  863              824   
                 Non-Dasher        9939                 3676             3100   
Jan1-Apr27-2020  All              65688                24354            20914   
Jun14-Jul03-2019 Breezer           1512                  314              467   
Sep18-Oct07-2019 Mizzle            4158                 1047             1231   

IS_NEW_CUSTOMER                               False                       \
MOSAIC                      Booming with Confidence Flourishing Families   
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher                        3476                 1756   
                 Non-Dasher                    5140                 2588   
Jan1-Apr27-2020  All                          28577                14422   
Jun14-Jul03-2019 Breezer                       1648                  761   
Sep18-Oct07-2019 Mizzle                        2420                 1174   

IS_NEW_CUSTOMER                                                                
MOSAIC                      Power Elite Singles and Starters Young City Solos  
TIMELINE         PRODUCT                                                       
Apr28-May16-2020 Dasher           10270                 1822             2450  
                 Non-Dasher       12601                 2690             3310  
Jan1-Apr27-2020  All              70652                14330            17147  
Jun14-Jul03-2019 Breezer           3717                  443              811  
Sep18-Oct07-2019 Mizzle            7107                 1129             1719

### All customers broken down by product, timeline and New/Existing status + GENDER

In [160]:
dashers_gender_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
a.is_new_customer,
left(c.I1GENDERCODE, 1) as customer_gender,
count(distinct a.customer_id) as num_customers
from
fact_sales as a
left join
dim_customer as b
on a.customer_id = b.id
left join
experian_data as c
on b.email = c.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [161]:
dashers_gender = pd.DataFrame(cs.execute(dashers_gender_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [162]:
dashers_gender['NUM_CUSTOMERS'] = pd.to_numeric(dashers_gender['NUM_CUSTOMERS'])

In [166]:
dashers_gender['PRODUCT'] = np.where((dashers_gender['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_gender['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_gender['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_gender['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_gender['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_gender['TIMELINE'] == 'Apr28-May16-2020')&(dashers_gender['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_gender['TIMELINE'] == 'Apr28-May16-2020')&(dashers_gender['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [167]:
dashers_gender = dashers_gender[~dashers_gender['PRODUCT'].isin(['Rest'])]

In [170]:
dashers_gender.groupby('CUSTOMER_GENDER')['NUM_CUSTOMERS'].sum()

CUSTOMER_GENDER
         17
B       229
F    328405
M    231803
U     20369
Name: NUM_CUSTOMERS, dtype: int64

In [174]:
dashers_gender_small = dashers_gender[dashers_gender['CUSTOMER_GENDER'].isin(['F','M'])]

In [175]:
dashers_gender_new_pivot = dashers_gender_small[dashers_gender_small['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','CUSTOMER_GENDER'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [176]:
dashers_gender_old_pivot = dashers_gender_small[dashers_gender_small['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','CUSTOMER_GENDER'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [179]:
dashers_gender_pivot = pd.merge(dashers_gender_new_pivot, dashers_gender_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_gender_pivot
dashers_gender_pivot.to_clipboard()

IS_NEW_CUSTOMER               True           False       
CUSTOMER_GENDER                   F      M       F      M
TIMELINE         PRODUCT                                 
Apr28-May16-2020 Dasher        4764   4863   11596  13327
                 Non-Dasher   19639  14834   19836  14718
Jan1-Apr27-2020  All         135194  89419  110713  78041
Jun14-Jul03-2019 Breezer       4170    523    8604   1158
Sep18-Oct07-2019 Mizzle        5692   6207    8197   8713

### All customers broken down by product, timeline and New/Existing status + AGE

In [181]:
dashers_age_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
a.is_new_customer,
case
when try_cast(right(c.i1combinedage, 2) as integer) <= 25 then 'a.18-25'
when try_cast(right(c.i1combinedage, 2) as integer) <= 35 then 'b.26-35'
when try_cast(right(c.i1combinedage, 2) as integer) <= 45 then 'c.36-45'
when try_cast(right(c.i1combinedage, 2) as integer) <= 55 then 'd.46-55'
when try_cast(right(c.i1combinedage, 2) as integer) <= 65 then 'e.56-65'
when try_cast(right(c.i1combinedage, 2) as integer) > 65 then 'f.65+'
end
as age_group,
count(distinct a.customer_id) as num_customers
from
fact_sales as a
left join
dim_customer as b
on a.customer_id = b.id
left join
experian_data as c
on b.email = c.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [182]:
dashers_age = pd.DataFrame(cs.execute(dashers_age_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [183]:
dashers_age['NUM_CUSTOMERS'] = pd.to_numeric(dashers_age['NUM_CUSTOMERS'])

In [185]:
dashers_age['PRODUCT'] = np.where((dashers_age['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_age['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_age['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_age['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_age['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_age['TIMELINE'] == 'Apr28-May16-2020')&(dashers_age['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_age['TIMELINE'] == 'Apr28-May16-2020')&(dashers_age['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [187]:
dashers_age = dashers_age[~dashers_age['PRODUCT'].isin(['Rest'])]

In [195]:
dashers_age_small = dashers_age[dashers_age['AGE_GROUP'].isin(['a.18-25', 'b.26-35', 'c.36-45', 'd.46-55', 'e.56-65', 'f.65+'])]

In [196]:
dashers_age_new_pivot = dashers_age_small[dashers_age_small['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [197]:
dashers_age_old_pivot = dashers_age_small[dashers_age_small['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [200]:
dashers_age_pivot = pd.merge(dashers_age_new_pivot, dashers_age_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_age_pivot
dashers_age_pivot.to_clipboard()

IS_NEW_CUSTOMER               True                                          \
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65  f.65+   
TIMELINE         PRODUCT                                                     
Apr28-May16-2020 Dasher        1186    2525    1214    1498    1442    900   
                 Non-Dasher    3839   10083    4731    4682    4643   3659   
Jan1-Apr27-2020  All          22882   66578   31427   30742   30458  24626   
Jun14-Jul03-2019 Breezer        217    1281    1040     650     644    611   
Sep18-Oct07-2019 Mizzle         924    3135    1775    1957    1881   1188   

IS_NEW_CUSTOMER               False                                         
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65  f.65+  
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher        2023    6260    3693    4119    4150   2805  
                 Non-Dasher    2733    9359    4724    5257    5758   4105  
Jan1-Apr27-2020  All          13953   49311   26434   29294   32482  23311  
Jun14-Jul03-2019 Breezer        352    2031    1660    1503    1845   1839  
Sep18-Oct07-2019 Mizzle        1011    4073    2329    2812    3190   2228

### All customers broken down by product, timeline and New/Existing status + INCOME

In [201]:
dashers_income_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
a.is_new_customer,
c.ESTIMATEDINCOMERANGEV6 as income_group,
count(distinct a.customer_id) as num_customers
from
fact_sales as a
left join
dim_customer as b
on a.customer_id = b.id
left join
experian_data as c
on b.email = c.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [202]:
dashers_income = pd.DataFrame(cs.execute(dashers_income_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [203]:
dashers_income['NUM_CUSTOMERS'] = pd.to_numeric(dashers_income['NUM_CUSTOMERS'])

In [204]:
dashers_income['PRODUCT'] = np.where((dashers_income['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_income['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_income['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_income['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_income['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_income['TIMELINE'] == 'Apr28-May16-2020')&(dashers_income['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_income['TIMELINE'] == 'Apr28-May16-2020')&(dashers_income['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [205]:
dashers_income = dashers_income[~dashers_income['PRODUCT'].isin(['Rest'])]

In [208]:
income_group = pd.read_excel("experian_dictionary.xlsx", sheet_name="income")

In [209]:
dashers_income = pd.merge(dashers_income, income_group, how = 'left', on = 'INCOME_GROUP')

In [217]:
dashers_income.groupby(['INCOME_GROUP', 'INCOME'])['NUM_CUSTOMERS'].sum()

INCOME_GROUP  INCOME            
A             $1,000-$14,999         13229
B             $15,000-$24,999        12172
C             $25,000-$34,999        14375
D             $35,000-$49,999        29710
E             $50,000-$74,999        64042
F             $75,000-$99,999        76762
G             $100,000-$124,999      69348
H             $125,000-$149,999      53033
I             $150,000-$174,999      36363
J             $175,000-$199,999      35715
K             $200,000-$249,999      69302
L             $250,000+             105300
U             Unknown                  578
Name: NUM_CUSTOMERS, dtype: int64

In [221]:
dashers_income['INCOME_GROUP_SMALL'] = np.where(dashers_income['INCOME_GROUP'].isin(['A','B','C','D']), 'A+<$50k',
                                               dashers_income['INCOME_GROUP']+'+'+dashers_income['INCOME'])

In [223]:
dashers_income_small = dashers_income[dashers_income['INCOME_GROUP'].isin(['A', 'B', 'C', 'D', 'E', 
                                                                           'F', 'G', 'H', 'I', 'J', 'K', 'L'])]

In [227]:
dashers_income_new_pivot = dashers_income_small[dashers_income_small['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','INCOME_GROUP_SMALL'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [228]:
dashers_income_old_pivot = dashers_income_small[dashers_income_small['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','INCOME_GROUP_SMALL'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum')

In [229]:
dashers_income_new_pivot

IS_NEW_CUSTOMER                True                                        \
INCOME_GROUP_SMALL          A+<$50k E+$50,000-$74,999  F+$75,000-$99,999    
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher        1150               1112               1284   
                 Non-Dasher    4972               4372               4918   
Jan1-Apr27-2020  All          30971              27968              32820   
Jun14-Jul03-2019 Breezer        581                550                652   
Sep18-Oct07-2019 Mizzle        1377               1290               1643   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          G+$100,000-$124,999  H+$125,000-$149,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                     1088                  923   
                 Non-Dasher                 4353                 3468   
Jan1-Apr27-2020  All                       29148                22319   
Jun14-Jul03-2019 Breezer                     598                  479   
Sep18-Oct07-2019 Mizzle                     1421                 1127   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          I+$150,000-$174,999  J+$175,000-$199,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                      596                  657   
                 Non-Dasher                 2220                 2225   
Jan1-Apr27-2020  All                       14718                14358   
Jun14-Jul03-2019 Breezer                     352                  296   
Sep18-Oct07-2019 Mizzle                      760                  816   

IS_NEW_CUSTOMER                                                
INCOME_GROUP_SMALL          K+$200,000-$249,999  L+$250,000+   
TIMELINE         PRODUCT                                       
Apr28-May16-2020 Dasher                     1263         1890  
                 Non-Dasher                 3933         5389  
Jan1-Apr27-2020  All                       24872        36168  
Jun14-Jul03-2019 Breezer                     568          762  
Sep18-Oct07-2019 Mizzle                     1584         2298

In [230]:
dashers_income_pivot = pd.merge(dashers_income_new_pivot, dashers_income_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_income_pivot
dashers_income_pivot.to_clipboard()

IS_NEW_CUSTOMER               True                                         \
INCOME_GROUP_SMALL          A+<$50k E+$50,000-$74,999  F+$75,000-$99,999    
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher        1150               1112               1284   
                 Non-Dasher    4972               4372               4918   
Jan1-Apr27-2020  All          30971              27968              32820   
Jun14-Jul03-2019 Breezer        581                550                652   
Sep18-Oct07-2019 Mizzle        1377               1290               1643   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          G+$100,000-$124,999  H+$125,000-$149,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                     1088                  923   
                 Non-Dasher                 4353                 3468   
Jan1-Apr27-2020  All                       29148                22319   
Jun14-Jul03-2019 Breezer                     598                  479   
Sep18-Oct07-2019 Mizzle                     1421                 1127   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          I+$150,000-$174,999  J+$175,000-$199,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                      596                  657   
                 Non-Dasher                 2220                 2225   
Jan1-Apr27-2020  All                       14718                14358   
Jun14-Jul03-2019 Breezer                     352                  296   
Sep18-Oct07-2019 Mizzle                      760                  816   

IS_NEW_CUSTOMER                                                 False  \
INCOME_GROUP_SMALL          K+$200,000-$249,999  L+$250,000+  A+<$50k   
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                     1263         1890    2505   
                 Non-Dasher                 3933         5389    3983   
Jan1-Apr27-2020  All                       24872        36168   21128   
Jun14-Jul03-2019 Breezer                     568          762    1114   
Sep18-Oct07-2019 Mizzle                     1584         2298    1705   

IS_NEW_CUSTOMER                                                    \
INCOME_GROUP_SMALL          E+$50,000-$74,999  F+$75,000-$99,999    
TIMELINE         PRODUCT                                            
Apr28-May16-2020 Dasher                   2475               3111   
                 Non-Dasher               3679               4576   
Jan1-Apr27-2020  All                     19921              24431   
Jun14-Jul03-2019 Breezer                   976               1229   
Sep18-Oct07-2019 Mizzle                   1699               2098   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          G+$100,000-$124,999  H+$125,000-$149,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                     2796                 2201   
                 Non-Dasher                 4142                 3180   
Jan1-Apr27-2020  All                       22858                17062   
Jun14-Jul03-2019 Breezer                    1134                  880   
Sep18-Oct07-2019 Mizzle                     1810                 1394   

IS_NEW_CUSTOMER                                                        \
INCOME_GROUP_SMALL          I+$150,000-$174,999  J+$175,000-$199,999    
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                     1568                 1548   
                 Non-Dasher                 2193                 2197   
Jan1-Apr27-2020  All                       12221                12032   
Jun14-Jul03-2019 Breezer                     689    

### All customers broken down by product, timeline and New/Existing status + SHIPPING REGION

In [232]:
dashers_shipping_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
a.is_new_customer,
a.shipping_reigion,
count(distinct a.customer_id) as num_customers
from
fact_sales as a
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [233]:
dashers_shipping = pd.DataFrame(cs.execute(dashers_shipping_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [234]:
dashers_shipping['NUM_CUSTOMERS'] = pd.to_numeric(dashers_shipping['NUM_CUSTOMERS'])

In [237]:
dashers_shipping['PRODUCT'] = np.where((dashers_shipping['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_shipping['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_shipping['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_shipping['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_shipping['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_shipping['TIMELINE'] == 'Apr28-May16-2020')&(dashers_shipping['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_shipping['TIMELINE'] == 'Apr28-May16-2020')&(dashers_shipping['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [238]:
dashers_shipping = dashers_shipping[~dashers_shipping['PRODUCT'].isin(['Rest'])]

In [242]:
dashers_shipping.groupby('SHIPPING_REIGION')['NUM_CUSTOMERS'].sum().sort_values(ascending = False).index

Index(['California', 'New York', 'Texas', 'Massachusetts', 'Illinois',
       'Florida', 'Pennsylvania', 'New Jersey', 'Virginia', 'North Carolina',
       'Washington', 'Ohio', 'Georgia', 'Colorado', 'Maryland', 'Michigan',
       'Minnesota', 'Connecticut', 'Tennessee', 'Arizona', 'Oregon',
       'Missouri', 'Wisconsin', 'Indiana', 'South Carolina',
       'District of Columbia', 'Utah', 'Kentucky', 'Kansas', 'Alabama',
       'Oklahoma', 'Louisiana', 'Iowa', 'New Hampshire', 'Nevada',
       'Rhode Island', 'Maine', 'Nebraska', 'Arkansas', 'Delaware', 'Idaho',
       'New Mexico', 'Hawaii', 'Vermont', 'Montana', 'Mississippi',
       'West Virginia', 'Alaska', 'South Dakota', 'North Dakota', 'Wyoming',
       'Armed Forces Europe', 'Armed Forces Pacific', 'Puerto Rico',
       'Virgin Islands', 'Guam', 'Armed Forces Americas', 'REDACTED', 'Palau'],
      dtype='object', name='SHIPPING_REIGION')

In [243]:
dashers_shipping_small = dashers_shipping[dashers_shipping['SHIPPING_REIGION'].isin(['California', 'New York', 
                                                                                     'Texas', 'Massachusetts',
                                                                                     'Illinois','Florida', 'Pennsylvania', 
                                                                                     'New Jersey', 'Virginia', 'Washington'])]

In [262]:
dashers_shipping_new_pivot = dashers_shipping_small[dashers_shipping_small['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','SHIPPING_REIGION'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum').sort_values(by = ('Jan1-Apr27-2020', 'All'), axis = 1, ascending = False)

In [263]:
dashers_shipping_old_pivot = dashers_shipping_small[dashers_shipping_small['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','SHIPPING_REIGION'], 
                                 values =  'NUM_CUSTOMERS',
                                 aggfunc = 'sum').sort_values(by = ('Jan1-Apr27-2020', 'All'), axis = 1, ascending = False)

In [266]:
dashers_shipping_pivot = pd.merge(dashers_shipping_new_pivot, dashers_shipping_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_shipping_pivot
dashers_shipping_pivot.to_clipboard()

IS_NEW_CUSTOMER                  True                                         \
SHIPPING_REIGION            California New York  Texas Florida Massachusetts   
TIMELINE         PRODUCT                                                       
Apr28-May16-2020 Dasher           1528      693    671     554           524   
                 Non-Dasher       5057     2805   1873    1383          2046   
Jan1-Apr27-2020  All             37678    16152  15092   11099         10949   
Jun14-Jul03-2019 Breezer           952      441    258     183           339   
Sep18-Oct07-2019 Mizzle           1755     1281    559     276           705   

IS_NEW_CUSTOMER                                                        \
SHIPPING_REIGION            Illinois Pennsylvania Virginia New Jersey   
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher          499          418      351        427   
                 Non-Dasher     1973         1640     1289       1425   
Jan1-Apr27-2020  All           10778        10265     9073       8209   
Jun14-Jul03-2019 Breezer         228          156      221        161   
Sep18-Oct07-2019 Mizzle          779          505      394        519   

IS_NEW_CUSTOMER                             False                          \
SHIPPING_REIGION            Washington California New York  Texas Florida   
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher            308       4482     1852   1822    1384   
                 Non-Dasher       1175       5390     2866   1939    1546   
Jan1-Apr27-2020  All              6813      34153    14813  12754   10679   
Jun14-Jul03-2019 Breezer           168       2034      919    548     375   
Sep18-Oct07-2019 Mizzle            747       2695     2225    791     392   

IS_NEW_CUSTOMER                                                             \
SHIPPING_REIGION            Massachusetts Illinois Pennsylvania New Jersey   
TIMELINE         PRODUCT                                                     
Apr28-May16-2020 Dasher              1285     1417          988        972   
                 Non-Dasher          2073     1947         1605       1576   
Jan1-Apr27-2020  All                 9404     8964         8209       7786   
Jun14-Jul03-2019 Breezer              541      451          355        383   
Sep18-Oct07-2019 Mizzle              1028     1065          682        811   

IS_NEW_CUSTOMER                                  
SHIPPING_REIGION            Virginia Washington  
TIMELINE         PRODUCT                         
Apr28-May16-2020 Dasher          848        771  
                 Non-Dasher     1335       1156  
Jan1-Apr27-2020  All            7713       5427  
Jun14-Jul03-2019 Breezer         412        367  
Sep18-Oct07-2019 Mizzle          535       1077

### All customers broken down by Breezer and Dasher post launch + WARDROBING SIZES

In [273]:
dashers_sizewardrobing_query = """
select product_timeline, num_sizes, count(distinct order_name) as num_orders 
from
(
select 
case 
when (a.happened_at_local_date between '2020-04-28' and '2020-05-16') and (a.taxonomy_style = 'Tree Dasher') then 'Dasher - Apr28-May16-2020' 
when (a.happened_at_local_date between '2019-06-14' and '2019-07-03') and (a.taxonomy_style = 'Tree Breezer') then 'Breezer - Jun14-Jul03-2019'
end as product_timeline,
order_name,
count(distinct size_us) as num_sizes
from
fact_sales as a
where 
a.taxonomy_style in ('Tree Breezer', 'Tree Dasher')
and
((a.happened_at_local_date between '2020-04-28' and '2020-05-16')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2
order by 1,2
)
group by 1, 2
order by 1, 2
"""

In [274]:
dashers_sizewardrobing = pd.DataFrame(cs.execute(dashers_sizewardrobing_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [278]:
dashers_sizes_pivot = dashers_sizewardrobing.pivot_table(index = 'NUM_SIZES', columns = 'PRODUCT_TIMELINE',
                                  values = 'NUM_ORDERS', aggfunc = 'sum')
dashers_sizes_pivot
dashers_sizes_pivot.to_clipboard()

PRODUCT_TIMELINE,Breezer - Jun14-Jul03-2019,Dasher - Apr28-May16-2020
NUM_SIZES,,
1,14885.0,36191.0
2,925.0,2238.0
3,60.0,141.0
4,7.0,32.0
5,2.0,8.0
6,2.0,5.0
8,1.0,NaN
13,NaN,1.0


### All customers broken down by Breezer and Dasher post launch + WARDROBING COLORS

In [279]:
dashers_colorwardrobing_query = """
select product_timeline, num_colors, count(distinct order_name) as num_orders 
from
(
select 
case 
when (a.happened_at_local_date between '2020-04-28' and '2020-05-16') and (a.taxonomy_style = 'Tree Dasher') then 'Dasher - Apr28-May16-2020' 
when (a.happened_at_local_date between '2019-06-14' and '2019-07-03') and (a.taxonomy_style = 'Tree Breezer') then 'Breezer - Jun14-Jul03-2019'
end as product_timeline,
order_name,
count(distinct color_name) as num_colors
from
fact_sales as a
where 
a.taxonomy_style in ('Tree Breezer', 'Tree Dasher')
and
((a.happened_at_local_date between '2020-04-28' and '2020-05-16')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
group by 1,2
order by 1,2
)
group by 1, 2
order by 1, 2
"""

In [280]:
dashers_colorwardrobing = pd.DataFrame(cs.execute(dashers_colorwardrobing_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [281]:
dashers_colors_pivot = dashers_colorwardrobing.pivot_table(index = 'NUM_COLORS', columns = 'PRODUCT_TIMELINE',
                                  values = 'NUM_ORDERS', aggfunc = 'sum')
dashers_colors_pivot
dashers_colors_pivot.to_clipboard()

PRODUCT_TIMELINE,Breezer - Jun14-Jul03-2019,Dasher - Apr28-May16-2020
NUM_COLORS,,
1,14732.0,36727.0
2,1042.0,1785.0
3,94.0,89.0
4,10.0,15.0
5,4.0,NaN


### All customers broken down by product, timeline and New/Existing status + Channel

In [282]:
dashers_channel_query = """
select * from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
case when c.channel_grouping in ('Social', 'Paid Social') then 'Social' else channel_grouping end as channel,
a.is_new_customer,
count(distinct a.order_name) as num_orders
from
fact_sales as a
left join
fivetran.google_analytics_360.session_hit as b
on a.order_name = b.transaction_transaction_id
left join
fivetran.google_analytics_360.ga_session as c
on
b.visit_id = c.visit_id
and b.visitor_id = c.visitor_id
and b.visit_start_time = c.visit_start_time
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
and a.event_type = 'order'
group by 1,2,3,4
order by 1,2,3,4
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
"""

In [283]:
dashers_channel = pd.DataFrame(cs.execute(dashers_channel_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [286]:
dashers_channel['NUM_ORDERS'] = pd.to_numeric(dashers_channel['NUM_ORDERS'])

In [287]:
dashers_channel['PRODUCT'] = np.where((dashers_channel['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_channel['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_channel['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_channel['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_channel['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_channel['TIMELINE'] == 'Apr28-May16-2020')&(dashers_channel['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_channel['TIMELINE'] == 'Apr28-May16-2020')&(dashers_channel['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [288]:
dashers_channel = dashers_channel[~dashers_channel['PRODUCT'].isin(['Rest'])]

In [298]:
dashers_channel_new_pivot = dashers_channel[dashers_channel['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','CHANNEL'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum').sort_values(by = ('Jan1-Apr27-2020', 'All'), axis = 1, ascending = False)

In [299]:
dashers_channel_old_pivot = dashers_channel[dashers_channel['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','CHANNEL'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum').sort_values(by = ('Jan1-Apr27-2020', 'All'), axis = 1, ascending = False)

In [300]:
dashers_channel_pivot = pd.merge(dashers_channel_new_pivot, dashers_channel_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_channel_pivot
dashers_channel_pivot.T.to_clipboard()

IS_NEW_CUSTOMER                True                                       \
CHANNEL                       Direct Organic Search Paid Search   Social   
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher       2169.0         2570.0      1530.0   1537.0   
                 Non-Dasher   7916.0        12346.0      5382.0    943.0   
Jan1-Apr27-2020  All         51688.0        49798.0     47134.0  15794.0   
Jun14-Jul03-2019 Breezer       707.0          623.0       718.0    785.0   
Sep18-Oct07-2019 Mizzle       3707.0         2098.0      2978.0   1319.0   

IS_NEW_CUSTOMER                                                           \
CHANNEL                        Email Affiliates Display Referral (Other)   
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher        664.0      562.0   125.0    215.0    23.0   
                 Non-Dasher    698.0     2030.0   891.0    783.0    44.0   
Jan1-Apr27-2020  All         15554.0     7651.0  7622.0   4284.0   269.0   
Jun14-Jul03-2019 Breezer      1205.0      375.0    65.0     87.0    18.0   
Sep18-Oct07-2019 Mizzle        324.0      510.0   449.0    207.0     3.0   

IS_NEW_CUSTOMER                                  False                 \
CHANNEL                     Sponsored Content   Direct Organic Search   
TIMELINE         PRODUCT                                                
Apr28-May16-2020 Dasher                   NaN   5508.0         4938.0   
                 Non-Dasher               NaN   8495.0        10247.0   
Jan1-Apr27-2020  All                      8.0  45440.0        35164.0   
Jun14-Jul03-2019 Breezer                  NaN   1201.0          805.0   
Sep18-Oct07-2019 Mizzle                   NaN   6838.0         1827.0   

IS_NEW_CUSTOMER                                                            \
CHANNEL                        Email Paid Search  Social Display Referral   
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher       9984.0      1823.0  2576.0   121.0    284.0   
                 Non-Dasher   5258.0      3191.0   764.0   386.0    488.0   
Jan1-Apr27-2020  All         31777.0     30203.0  7074.0  2836.0   2555.0   
Jun14-Jul03-2019 Breezer      6302.0       979.0   262.0   119.0     93.0   
Sep18-Oct07-2019 Mizzle       3539.0      2675.0   704.0   398.0    177.0   

IS_NEW_CUSTOMER                                                   
CHANNEL                     Affiliates (Other) Sponsored Content  
TIMELINE         PRODUCT                                          
Apr28-May16-2020 Dasher          529.0    25.0               NaN  
                 Non-Dasher      696.0    35.0               NaN  
Jan1-Apr27-2020  All            2362.0   175.0               7.0  
Jun14-Jul03-2019 Breezer         231.0    32.0               NaN  
Sep18-Oct07-2019 Mizzle          363.0    15.0               NaN

### All customers broken down by product, timeline and New/Existing status + Channel

In [303]:
dashers_social_query = """
select
timeline,
taxonomy_style,
is_new_customer,
case 
when traffic_source_source ilike '%facebook%' then 'facebook'
when traffic_source_source ilike '%instagram%' then 'instagram'
else 'others'
end as social,
count(distinct order_name) as num_orders
from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
case when c.channel_grouping in ('Social', 'Paid Social') then 'Social' else channel_grouping end as channel,
c.traffic_source_source,
a.is_new_customer,
a.order_name
from
fact_sales as a
left join
fivetran.google_analytics_360.session_hit as b
on a.order_name = b.transaction_transaction_id
left join
fivetran.google_analytics_360.ga_session as c
on
b.visit_id = c.visit_id
and b.visitor_id = c.visitor_id
and b.visit_start_time = c.visit_start_time
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
and a.event_type = 'order'
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
and channel in ('Social')
group by 1, 2, 3, 4
order by 1, 2, 3, 4
"""

In [304]:
dashers_social = pd.DataFrame(cs.execute(dashers_social_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [307]:
dashers_social['NUM_ORDERS'] = pd.to_numeric(dashers_social['NUM_ORDERS'])

In [308]:
dashers_social['PRODUCT'] = np.where((dashers_social['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_social['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_social['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_social['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_social['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_social['TIMELINE'] == 'Apr28-May16-2020')&(dashers_social['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_social['TIMELINE'] == 'Apr28-May16-2020')&(dashers_social['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [309]:
dashers_social = dashers_social[~dashers_social['PRODUCT'].isin(['Rest'])]

In [316]:
dashers_social_new_pivot = dashers_social[dashers_social['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','SOCIAL'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [317]:
dashers_social_old_pivot = dashers_social[dashers_social['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','SOCIAL'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [318]:
dashers_social_pivot = pd.merge(dashers_social_new_pivot, dashers_social_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_social_pivot
dashers_social_pivot.T.to_clipboard()

IS_NEW_CUSTOMER                True                      False            \
SOCIAL                      facebook instagram others facebook instagram   
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher         1447        33     57     2393        86   
                 Non-Dasher      835        35     73      688        36   
Jan1-Apr27-2020  All           14354       386   1054     6409       183   
Jun14-Jul03-2019 Breezer         702        42     41      184        44   
Sep18-Oct07-2019 Mizzle         1237        28     54      640        31   

IS_NEW_CUSTOMER                     
SOCIAL                      others  
TIMELINE         PRODUCT            
Apr28-May16-2020 Dasher         97  
                 Non-Dasher     40  
Jan1-Apr27-2020  All           482  
Jun14-Jul03-2019 Breezer        34  
Sep18-Oct07-2019 Mizzle         33

### New customers broken down by product, timeline for customers from Social Channel + Facebook

In [320]:
dashers_facebook_query = """
select
timeline,
taxonomy_style,
is_new_customer,
age_group,
count(distinct order_name) as num_orders
from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
c.traffic_source_source,
a.is_new_customer,
a.order_name,
case
when try_cast(right(e.i1combinedage, 2) as integer) <= 25 then 'a.18-25'
when try_cast(right(e.i1combinedage, 2) as integer) <= 35 then 'b.26-35'
when try_cast(right(e.i1combinedage, 2) as integer) <= 45 then 'c.36-45'
when try_cast(right(e.i1combinedage, 2) as integer) <= 55 then 'd.46-55'
when try_cast(right(e.i1combinedage, 2) as integer) <= 65 then 'e.56-65'
when try_cast(right(e.i1combinedage, 2) as integer) > 65 then 'f.65+'
end
as age_group
from
fact_sales as a
left join
fivetran.google_analytics_360.session_hit as b
on a.order_name = b.transaction_transaction_id
left join
fivetran.google_analytics_360.ga_session as c
on
b.visit_id = c.visit_id
and b.visitor_id = c.visitor_id
and b.visit_start_time = c.visit_start_time
left join
dim_customer as d
on a.customer_id = d.id
left join
experian_data as e
on d.email = e.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
and a.event_type = 'order'
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
and traffic_source_source ilike '%facebook%'
group by 1, 2, 3, 4
order by 1, 2, 3, 4
"""

In [321]:
dashers_facebook = pd.DataFrame(cs.execute(dashers_facebook_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [322]:
dashers_facebook['NUM_ORDERS'] = pd.to_numeric(dashers_facebook['NUM_ORDERS'])

In [323]:
dashers_facebook['PRODUCT'] = np.where((dashers_facebook['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_facebook['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_facebook['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_facebook['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_facebook['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_facebook['TIMELINE'] == 'Apr28-May16-2020')&(dashers_facebook['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_facebook['TIMELINE'] == 'Apr28-May16-2020')&(dashers_facebook['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [324]:
dashers_facebook = dashers_facebook[~dashers_facebook['PRODUCT'].isin(['Rest'])]

In [331]:
dashers_facebook_new_pivot = dashers_facebook[dashers_facebook['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [332]:
dashers_facebook_old_pivot = dashers_facebook[dashers_facebook['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [333]:
dashers_facebook_pivot = pd.merge(dashers_facebook_new_pivot, dashers_facebook_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_facebook_pivot
dashers_facebook_pivot.T.to_clipboard()

IS_NEW_CUSTOMER               True                                         \
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65 f.65+   
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher         122     320     183     275     254   128   
                 Non-Dasher      65     218     133     107     115    91   
Jan1-Apr27-2020  All           1039    3376    2050    2153    2246  2075   
Jun14-Jul03-2019 Breezer         21     183     186     101      96    61   
Sep18-Oct07-2019 Mizzle          84     345     175     241     184    65   

IS_NEW_CUSTOMER               False                                        
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65 f.65+  
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher         192     623     401     405     349   191  
                 Non-Dasher      49     169     111     119     102    69  
Jan1-Apr27-2020  All            380    1399     891    1032    1356   791  
Jun14-Jul03-2019 Breezer          6      40      43      34      28    18  
Sep18-Oct07-2019 Mizzle          40     196     101     116     101    24

### New customers broken down by product, timeline for customers from Social Channel + Instagram

In [334]:
dashers_instagram_query = """
select
timeline,
taxonomy_style,
is_new_customer,
age_group,
count(distinct order_name) as num_orders
from
(
select 
case 
when a.happened_at_local_date between '2020-01-01' and '2020-04-27' then 'Jan1-Apr27-2020' 
when a.happened_at_local_date between '2020-04-28' and '2020-05-16' then 'Apr28-May16-2020' 
when a.happened_at_local_date between '2019-09-18' and '2019-10-07' then 'Sep18-Oct07-2019'
when a.happened_at_local_date between '2019-06-14' and '2019-07-03' then 'Jun14-Jul03-2019'
end as timeline,
a.taxonomy_style,
c.traffic_source_source,
a.is_new_customer,
a.order_name,
case
when try_cast(right(e.i1combinedage, 2) as integer) <= 25 then 'a.18-25'
when try_cast(right(e.i1combinedage, 2) as integer) <= 35 then 'b.26-35'
when try_cast(right(e.i1combinedage, 2) as integer) <= 45 then 'c.36-45'
when try_cast(right(e.i1combinedage, 2) as integer) <= 55 then 'd.46-55'
when try_cast(right(e.i1combinedage, 2) as integer) <= 65 then 'e.56-65'
when try_cast(right(e.i1combinedage, 2) as integer) > 65 then 'f.65+'
end
as age_group
from
fact_sales as a
left join
fivetran.google_analytics_360.session_hit as b
on a.order_name = b.transaction_transaction_id
left join
fivetran.google_analytics_360.ga_session as c
on
b.visit_id = c.visit_id
and b.visitor_id = c.visitor_id
and b.visit_start_time = c.visit_start_time
left join
dim_customer as d
on a.customer_id = d.id
left join
experian_data as e
on d.email = e.email
where 
a.taxonomy_category = 'Shoes'
and
((a.happened_at_local_date between '2020-01-01' and '2020-05-16')
or (a.happened_at_local_date between '2019-09-18' and '2019-10-07')
or (a.happened_at_local_date between '2019-06-14' and '2019-07-03'))
and a.sales_channel = 'eCommerce'
and a.profit_center_country = 'United States'
and a.event_type = 'order'
)
where not (taxonomy_style = 'Dasher' and timeline = 'Jan1-Apr27-2020')
and traffic_source_source ilike '%instagram%'
group by 1, 2, 3, 4
order by 1, 2, 3, 4
"""

In [335]:
dashers_instagram = pd.DataFrame(cs.execute(dashers_instagram_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [336]:
dashers_instagram['NUM_ORDERS'] = pd.to_numeric(dashers_instagram['NUM_ORDERS'])

In [337]:
dashers_instagram['PRODUCT'] = np.where((dashers_instagram['TIMELINE'] == 'Jun14-Jul03-2019')&(dashers_instagram['TAXONOMY_STYLE'] == 'Tree Breezer'), 'Breezer',
                                          np.where((dashers_instagram['TIMELINE'] == 'Sep18-Oct07-2019')&(dashers_instagram['TAXONOMY_STYLE'].isin(['Wool Runner Mizzle', 'Wool Runner-up Mizzle'])), 'Mizzle',
                                                  np.where(dashers_instagram['TIMELINE'] == 'Jan1-Apr27-2020', 'All', 
                                                          np.where((dashers_instagram['TIMELINE'] == 'Apr28-May16-2020')&(dashers_instagram['TAXONOMY_STYLE'] == 'Tree Dasher'), 'Dasher',
                                                                  np.where((dashers_instagram['TIMELINE'] == 'Apr28-May16-2020')&(dashers_instagram['TAXONOMY_STYLE'] != 'Tree Dasher'), 'Non-Dasher', 'Rest')))))

In [338]:
dashers_instagram = dashers_instagram[~dashers_instagram['PRODUCT'].isin(['Rest'])]

In [339]:
dashers_instagram_new_pivot = dashers_instagram[dashers_instagram['IS_NEW_CUSTOMER'] == True].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [340]:
dashers_instagram_old_pivot = dashers_instagram[dashers_instagram['IS_NEW_CUSTOMER'] == False].pivot_table(index = ['TIMELINE','PRODUCT'], 
                                 columns = ['IS_NEW_CUSTOMER','AGE_GROUP'], 
                                 values =  'NUM_ORDERS',
                                 aggfunc = 'sum')

In [341]:
dashers_instagram_pivot = pd.merge(dashers_instagram_new_pivot, dashers_instagram_old_pivot, how = 'inner', on = ['TIMELINE', 'PRODUCT'])
dashers_instagram_pivot
dashers_instagram_pivot.T.to_clipboard()

IS_NEW_CUSTOMER               True                                         \
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65 f.65+   
TIMELINE         PRODUCT                                                    
Apr28-May16-2020 Dasher         5.0    10.0     3.0     6.0     1.0   NaN   
                 Non-Dasher     9.0    12.0     5.0     3.0     2.0   3.0   
Jan1-Apr27-2020  All           47.0   160.0    52.0    32.0    19.0  21.0   
Jun14-Jul03-2019 Breezer        2.0    15.0     8.0     5.0     4.0   NaN   
Sep18-Oct07-2019 Mizzle         6.0     8.0     2.0     1.0     2.0   NaN   

IS_NEW_CUSTOMER               False                                        
AGE_GROUP                   a.18-25 b.26-35 c.36-45 d.46-55 e.56-65 f.65+  
TIMELINE         PRODUCT                                                   
Apr28-May16-2020 Dasher          16      24       9       8      13     3  
                 Non-Dasher       8      13       4       4       2     1  
Jan1-Apr27-2020  All             20      58      29      25      25     8  
Jun14-Jul03-2019 Breezer          4      13      11       6       4     2  
Sep18-Oct07-2019 Mizzle           7       9       2       4       2     2